In [1]:
!pip install --upgrade  textblob gensim pytorch-nlp swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 16.6 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16506 sha256=ce45290d1a7fbef54abf457e81b28cca51768385541efed64d622fc989bea928
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully uninstalled textblob-0.17.1


In [2]:
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
import sys
from textblob import TextBlob, Word
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk

TRACE = False  # Setting to true is useful when debugging to know which device is being used
embedding_dim = 100
epochs=10

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)


set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
fi

if [ ! -f glove.6B.100d.txt ]; then
  wget -O glove.6B.100d.txt https://www.dropbox.com/s/dl1vswq2sz5f1ws/glove.6B.100d.txt?dl=0
fi

Writing get_data.sh


In [4]:
!bash get_data.sh

--2024-10-08 09:11:59--  https://www.dropbox.com/s/xds4lua69b7okw8/yelp.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/2k8im8ftu9yk8mnqhops9/yelp.csv?rlkey=52dzmxgys0su77wb6o75vb5ab&dl=0 [following]
--2024-10-08 09:11:59--  https://www.dropbox.com/scl/fi/2k8im8ftu9yk8mnqhops9/yelp.csv?rlkey=52dzmxgys0su77wb6o75vb5ab&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4224934a36a34c723bb057c124.dl.dropboxusercontent.com/cd/0/inline/CcAuyzvi5vVgsn3e1qOUNj5maxp9aOtcciGJ2ZYAz5XI-wMKYW5c7tKvB9aFKvAYkjgZys1OU5Y0epjlC41khCl8g6paujQVBQqRCZ0PA59yExrjOGYiVmYXpTzGYWQrEoclRqBkSBYH97lFVFF9Ymzb/file# [following]
--2024-10-08 09:12:00--  https://uc4224934a36a34c723bb057c124.dl.dropboxusercontent.com/cd/0/inlin

In [5]:
path = './yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
X = yelp_best_worst.text
y = yelp_best_worst.stars.map({1:0, 5:1})

In [6]:
yelp

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0


In [8]:
TextBlob(X.values[1]).correct()

TextBlob("I have no idea why some people give bad reviews about this place. It goes to show you, you can please everyone. They are probably gripping about something that their own fault...there are many people like that.

In any case, my friend and I arrived at about 5:50 of this past Sunday. It was pretty crowded, more than I thought for a Sunday evening and thought we would have to wait forever to get a seat but they said we'll be seated when the girl comes back from seating someone else. He were seated at 5:52 and the waiter came and got our drink orders. Everyone was very pleasant from the host that seated us to the waiter to the server. The prices were very good as well. He placed our orders once we decided what we wanted at 6:02. He shared the baked spaghetti alone and the small "Were's The Beef" penza so we can both try them. The alone was huge and we got the smallest one (personal) and got the small 11" penza. Both were awesome! By friend liked the penza better and I liked the 

In [9]:
# Create corpus of sentences such that the sentence has more than 3 words
corpus = [line for line in X.values if len(TextBlob(line).words)> 3]

In [10]:
corpus[:5]

['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!',
 'I have no idea why some people give bad reviews about this place. It goes to show you, you can

In [11]:
import itertools
from torchnlp.encoders import LabelEncoder


list_of_words = list(itertools.chain.from_iterable([sentence.split() for sentence in corpus]))
ids_from_words = LabelEncoder(list_of_words, reserved_labels=['UNK'], unknown_index=0, min_occurrences=1)


# tokenizer = Tokenizer()
# # Use the fit_on_texts method to fit the tokenizer
# tokenizer.fit_on_texts(corpus) # Fill

print(f'Before the tokenizer: {corpus[:1]}')

#Now use the same "trained" tokenizer to convert the corpus from words to IDs with the texts_to_sequences method
tokenized_corpus = [ids_from_words.batch_encode(sentence.split()) for sentence in corpus]

print(f'After the tokenizer: {tokenized_corpus[:1]}')

Before the tokenizer: ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!']
After the tokenizer: [tensor([  1,   2,   3,   4,   5,   6,   7,   8,   9,

In [12]:
def ids_from_text(text):
  return ids_from_words.batch_encode(text)

def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)

In [13]:
path_to_glove_file = "./glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400001 word vectors.


In [14]:
vocab_size = len(ids_from_words.vocab)

In [15]:
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in ids_from_words.token_to_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 13438 words (31786 misses)


In [16]:
def pad_sequence_of_tokens(x, maxlen, unk_token='UNK'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [17]:
# This is the algorithmic part of batching the dataset and yielding the window of words and expected middle word for each bacth as a generator.
def create_context_target_pairs(texts, context_size):
    data = []
    for text in texts:
        tokens = text.split()
        for i, word in enumerate(tokens):
            start = max(0, i - context_size)
            end = min(len(tokens), i + context_size + 1)
            context = pad_sequence_of_tokens([tokens[j] for j in range(start, end) if j != i], maxlen=4)
            target = ids_from_words.token_to_index[word]
            context_indices = [ids_from_words.token_to_index[w] for w in context]
            context_indices.append(target)
            data.append(torch.Tensor(context_indices))
    return data

In [18]:
data = create_context_target_pairs(corpus[:50], 2)

In [19]:
data = torch.stack(data)

In [20]:
X = data[:, :4].to(torch.long)
y = data[:, 4].to(torch.long)

In [21]:

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(embeddings=torch.Tensor(embedding_matrix).to(torch.float), freeze=True, padding_idx=0)
        # Linear layer to act as the hidden layer
        self.linear1 = nn.Linear(embedding_dim, 128)
        # Linear layer to predict the center word
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.mean(embeds, dim=1)
        out = torch.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.log_softmax(out, dim=1)
        return log_probs

In [22]:
def train_cbow(X, y, model, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0

        # Step 1. Recall that torch *accumulates* gradients. Before passing in a new instance,
        # you need to zero out the gradients from the old instance
        optimizer.zero_grad()

        # Step 2. Run the forward pass, getting log probabilities over next words
        log_probs = model(X)

        # Step 3. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, y)

        # Step 4. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Print progress
        if (epoch + 1) % 10 == 0:
            print('Epoch: {}, Loss: {:.4f}'.format(epoch + 1, total_loss))
    return model

In [23]:
context_size=2
embedding_dim=100
vocab_size = len(ids_from_words.vocab)
model = CBOW(vocab_size, embedding_dim, context_size * 2)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [24]:
trained_model = train_cbow(X, y, model, loss_function, optimizer, epochs=1)

In [25]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

embeddings = trained_model.embeddings.weight.data.cpu().numpy()

# Now, we need to save these embeddings in a format that gensim can understand
# For that, we will use the KeyedVectors instance in gensim

# Instantiate the KeyedVectors with the correct size
kv = KeyedVectors(vector_size=embeddings.shape[1])






In [26]:
# Add the vectors and their corresponding words to the KeyedVectors instance
kv.add_vectors(ids_from_words.index_to_token, embeddings)

In [27]:
kv.most_similar(positive=['gasoline'])

[('fuel', 0.8057637214660645),
 ('heating', 0.7722175717353821),
 ('prices', 0.7397462725639343),
 ('gallon', 0.7387636303901672),
 ('gas', 0.7218859195709229),
 ('oil', 0.6737465858459473),
 ('supply', 0.6719061136245728),
 ('barrels', 0.6625459790229797),
 ('supplies', 0.6523180603981018),
 ('demand', 0.6499200463294983)]

In [28]:
kv.most_similar(negative=['apple'])

[('**', 0.36834073066711426),
 ('wordly', 0.34334596991539),
 ('in-water', 0.34262025356292725),
 ('comping', 0.3354523181915283),
 ('acy', 0.3347332775592804),
 ('tinga', 0.32452431321144104),
 ('inbetween', 0.3242317736148834),
 ('burros', 0.32158100605010986),
 ('noches', 0.31810420751571655),
 ('peple', 0.3137488067150116)]

In [29]:
kv.most_similar(positive=['pizza'])

[('sandwich', 0.7488481402397156),
 ('sandwiches', 0.7221847772598267),
 ('bread', 0.700667679309845),
 ('pizzas', 0.7004502415657043),
 ('burger', 0.6977595686912537),
 ('snack', 0.6840099692344666),
 ('taco', 0.6788282990455627),
 ('pie', 0.6776915788650513),
 ('chicken', 0.6714670658111572),
 ('burgers', 0.66903156042099)]